# Applying Advanced Transformations (Core)

by Israel Diaz

## Assignment

I. Clean the files and combine them into one final DataFrame.

* This dataframe should have the following columns:
    * Hero (Just the name of the Hero)
    * Publisher
    * Gender
    * Eye color
    * Race
    * Hair color
    * Height (numeric)
    * Skin color
    * Alignment
    * Weight (numeric)
    * Plus, one-hot-encoded columns for every power that appears in the dataset. E.g.:
        * Agility
        * Flight
        * Superspeed
        * etc.

Hint: There is a space in "100 kg" or "52.5 cm"


II. Use your combined DataFrame to answer the following questions.

* Compare the average weight of super powers who have Super Speed to those who do not.
* What is the average height of heroes for each publisher?

## Solution

In [1]:
## load libraries
## Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
## Importing the OS and JSON Modules
import os,json


In [2]:
# Load Dataframes
sh_info = pd.read_csv('data/superhero_info - superhero_info.csv')
sh_powers = pd.read_csv('data/superhero_powers - superhero_powers.csv')

In [3]:
sh_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Hero|Publisher  463 non-null    object
 1   Gender          463 non-null    object
 2   Race            463 non-null    object
 3   Alignment       463 non-null    object
 4   Hair color      463 non-null    object
 5   Eye color       463 non-null    object
 6   Skin color      463 non-null    object
 7   Measurements    463 non-null    object
dtypes: object(8)
memory usage: 29.1+ KB


In [4]:
sh_powers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hero_names  667 non-null    object
 1   Powers      667 non-null    object
dtypes: object(2)
memory usage: 10.5+ KB


In [5]:
sh_info.head()

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


### Split Hero and Publisher

In [6]:
sh_info[['Hero','Publisher']] = sh_info['Hero|Publisher'].str.split('|',expand=True)
sh_info.head(2)

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


In [7]:
sh_info.drop(columns='Hero|Publisher', inplace=True)
sh_info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


### Strings into Dictionary

In [8]:
sh_info.Measurements[1]

"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"

In [9]:
## use .str.replace to replace all single quotes
sh_info['Measurements'] = sh_info['Measurements'].str.replace("'",'"')
## Apply the json.loads to the full column
sh_info['Measurements'] = sh_info['Measurements'].apply(json.loads)
sh_info['Measurements'].head()


0    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
1     {'Height': '191.0 cm', 'Weight': '65.0 kg'}
2     {'Height': '185.0 cm', 'Weight': '90.0 kg'}
3    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
4    {'Height': '193.0 cm', 'Weight': '122.0 kg'}
Name: Measurements, dtype: object

In [10]:
## unpack to separate columns
hei_wei = sh_info['Measurements'].apply(pd.Series)
hei_wei.head(5)

,Height,Weight
0,203.0 cm,441.0 kg
1,191.0 cm,65.0 kg
2,185.0 cm,90.0 kg
3,203.0 cm,441.0 kg
4,193.0 cm,122.0 kg


In [11]:
## Renaming columns name to include unit
hei_wei.columns = ['Height [cm]', 'Weight [Kg]']
hei_wei.head()

,Height [cm],Weight [Kg]
0,203.0 cm,441.0 kg
1,191.0 cm,65.0 kg
2,185.0 cm,90.0 kg
3,203.0 cm,441.0 kg
4,193.0 cm,122.0 kg


In [12]:
## convert to integer
for col in hei_wei.columns:
    hei_wei[col] = hei_wei[col].str.split(' ',expand=True)[0]
    hei_wei[col] = hei_wei[col].astype('float').astype('int')

In [13]:
hei_wei.head()

,Height [cm],Weight [Kg]
0,203,441
1,191,65
2,185,90
3,203,441
4,193,122


In [14]:
## Join both dataframe to one
sh_info = pd.concat((sh_info, hei_wei), axis=1)
sh_info.drop(columns='Measurements', inplace=True)
sh_info.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height [cm],Weight [Kg]
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203,441
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191,65
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185,90
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203,441
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193,122


### Organizing columns

In [15]:
sh_info = sh_info[['Hero', 'Publisher', 'Gender', 'Eye color', 'Race', 'Hair color', 'Height [cm]', 'Skin color', 'Alignment', 'Weight [Kg]']]

sh_info.head()

,Hero,Publisher,Gender,Eye color,Race,Hair color,Height [cm],Skin color,Alignment,Weight [Kg]
0,A-Bomb,Marvel Comics,Male,yellow,Human,No Hair,203,Unknown,good,441
1,Abe Sapien,Dark Horse Comics,Male,blue,Icthyo Sapien,No Hair,191,blue,good,65
2,Abin Sur,DC Comics,Male,blue,Ungaran,No Hair,185,red,good,90
3,Abomination,Marvel Comics,Male,green,Human / Radiation,No Hair,203,Unknown,bad,441
4,Absorbing Man,Marvel Comics,Male,blue,Human,No Hair,193,Unknown,bad,122


### OneHotEncoding Superpowers

In [16]:
## display superpowers dataframe
sh_powers.head()

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


In [17]:
## Transforming string into list
sh_powers['Pwr_split'] = sh_powers['Powers']
sh_powers['Pwr_split'] = sh_powers['Pwr_split'].str.split(',')
sh_powers.head()

,hero_names,Powers,Pwr_split
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed","[Agility, Super Strength, Stamina, Super Speed]"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...","[Accelerated Healing, Durability, Longevity, S..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du...","[Agility, Accelerated Healing, Cold Resistance..."
3,Abin Sur,Lantern Power Ring,[Lantern Power Ring]
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt...","[Accelerated Healing, Intelligence, Super Stre..."


In [18]:
## explode columns
exploded = sh_powers.explode('Pwr_split')
exploded

,hero_names,Powers,Pwr_split
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Agility
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Super Strength
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Stamina
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Super Speed
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...",Accelerated Healing
...,...,...,...
665,Zatanna,"Cryokinesis,Telepathy,Magic,Fire Control,Proba...",Weather Control
666,Zoom,"Super Speed,Intangibility,Time Travel,Time Man...",Super Speed
666,Zoom,"Super Speed,Intangibility,Time Travel,Time Man...",Intangibility
666,Zoom,"Super Speed,Intangibility,Time Travel,Time Man...",Time Travel


In [19]:
cols_to_make = exploded['Pwr_split'].dropna().unique()
cols_to_make

array(['Agility', 'Super Strength', 'Stamina', 'Super Speed',
       'Accelerated Healing', 'Durability', 'Longevity', 'Camouflage',
       'Self-Sustenance', 'Cold Resistance', 'Underwater breathing',
       'Marksmanship', 'Weapons Master', 'Intelligence', 'Telepathy',
       'Immortality', 'Reflexes', 'Enhanced Sight', 'Sub-Mariner',
       'Lantern Power Ring', 'Invulnerability', 'Animation',
       'Super Breath', 'Dimensional Awareness', 'Flight', 'Size Changing',
       'Teleportation', 'Magic', 'Dimensional Travel',
       'Molecular Manipulation', 'Energy Manipulation', 'Power Cosmic',
       'Energy Absorption', 'Elemental Transmogrification',
       'Fire Resistance', 'Natural Armor', 'Heat Resistance',
       'Matter Absorption', 'Regeneration', 'Stealth', 'Power Suit',
       'Energy Blasts', 'Energy Beams', 'Heat Generation', 'Danger Sense',
       'Phasing', 'Force Fields', 'Hypnokinesis', 'Invisibility',
       'Enhanced Senses', 'Jump', 'Shapeshifting', 'Elasticity',
 

In [20]:
## One Hot Encode
for col in cols_to_make:
    sh_powers[col] = sh_powers['Powers'].str.contains(col)
sh_powers.head()

C:\Users\diazi\AppData\Local\Temp\ipykernel_112244\3625071038.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sh_powers[col] = sh_powers['Powers'].str.contains(col)
C:\Users\diazi\AppData\Local\Temp\ipykernel_112244\3625071038.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sh_powers[col] = sh_powers['Powers'].str.contains(col)
C:\Users\diazi\AppData\Local\Temp\ipykernel_112244\3625071038.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has 

,hero_names,Powers,Pwr_split,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed","[Agility, Super Strength, Stamina, Super Speed]",True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...","[Accelerated Healing, Durability, Longevity, S...",False,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du...","[Agility, Accelerated Healing, Cold Resistance...",True,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,Lantern Power Ring,[Lantern Power Ring],False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt...","[Accelerated Healing, Intelligence, Super Stre...",False,True,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [21]:
sh_powers.drop(columns=['Powers', 'Pwr_split'], axis=1, inplace=True)
sh_powers.head()

,hero_names,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,Camouflage,Self-Sustenance,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,True,True,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,True,True,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,True,True,True,False,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,False,True,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [22]:
## Replace True with 1 and False with 0

boolean = {True: 1, False:0}
sh_powers.replace(boolean, inplace=True)
sh_powers.head()

,hero_names,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,Camouflage,Self-Sustenance,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A-Bomb,0,1,1,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,Abe Sapien,1,1,1,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Abin Sur,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Abomination,0,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Merge both dataframes

In [23]:
sh_powers = sh_info.merge(sh_powers, left_on='Hero', right_on='hero_names', how='inner').drop(columns='hero_names')

sh_powers.head(10)

,Hero,Publisher,Gender,Eye color,Race,Hair color,Height [cm],Skin color,Alignment,Weight [Kg],...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,A-Bomb,Marvel Comics,Male,yellow,Human,No Hair,203,Unknown,good,441,...,0,0,0,0,0,0,0,0,0,0
1,Abe Sapien,Dark Horse Comics,Male,blue,Icthyo Sapien,No Hair,191,blue,good,65,...,0,0,0,0,0,0,0,0,0,0
2,Abin Sur,DC Comics,Male,blue,Ungaran,No Hair,185,red,good,90,...,0,0,0,0,0,0,0,0,0,0
3,Abomination,Marvel Comics,Male,green,Human / Radiation,No Hair,203,Unknown,bad,441,...,0,0,0,0,0,0,0,0,0,0
4,Absorbing Man,Marvel Comics,Male,blue,Human,No Hair,193,Unknown,bad,122,...,0,0,0,0,0,0,0,0,0,0
5,Adam Strange,DC Comics,Male,blue,Human,Blond,185,Unknown,good,88,...,0,0,0,0,0,0,0,0,0,0
6,Agent Bob,Marvel Comics,Male,brown,Human,Brown,178,Unknown,good,81,...,0,0,0,0,0,0,0,0,0,0
7,Agent Zero,Marvel Comics,Male,Unknown,Unknown,Unknown,191,Unknown,good,104,...,0,0,0,0,0,0,0,0,0,0
8,Air-Walker,Marvel Comics,Male,blue,Unknown,White,188,Unknown,bad,108,...,0,0,0,0,0,0,0,0,0,0
9,Ajax,Marvel Comics,Male,brown,Cyborg,Black,193,Unknown,bad,90,...,0,0,0,0,0,0,0,0,0,0


### Comparing AVG weight of heros with superspeed

In [24]:
sh_powers[['Weight [Kg]', 'Super Speed']].groupby('Super Speed').mean()

,Weight [Kg]
Super Speed,
0,101.773585
1,129.404040


In average, Heros with super speed weights more than the others.

### Average Height for Publisher

In [25]:
sh_powers[['Publisher', 'Height [cm]']].groupby(by='Publisher').mean().sort_values(by='Height [cm]', ascending=False)

,Height [cm]
Publisher,
Image Comics,211.000000
Marvel Comics,191.545455
DC Comics,181.920290
Star Trek,181.500000
Team Epic TV,180.750000
Unknown,178.000000
Dark Horse Comics,176.909091
Shueisha,171.500000
George Lucas,159.600000


Taller Heros comes from Image Comics